In [ ]:
from typing import List
mat = [[8, 9], [10, 33]]

(a, b), (c, d) = mat[0], mat[1]
print(a, b, c, d)
det = a * d - b * c
assert det != 0, "det cannot be zero"
det = 2
inv_mat = [[d, -b], [-c, a]]
print([[item / det for item in inv_mat[j]] for j in range(2)])


def matrix_mul(mat1: List[List], mat2: List[List]) ->List[List]:
    assert len(mat1[0]) == len(mat2), "mat1's lines must be equal to mat2's rows"
    result = []
    for r in mat1:
        res_row = []
        for l in zip(*mat2):
            temp = [i * j for i, j in zip(r, l)]
            res_row.append(sum(temp))
        result.append(res_row)
    
    return result

print(matrix_mul(mat, [[5], [6]]))


In [ ]:
from concurrent.futures import ProcessPoolExecutor
import math
from multiprocessing import Pool, cpu_count


p_kn = []
p_kn.append(list(map(math.log, range(1, 10000))))

for i in range(3, 2):
    print(i)
# with Pool(cpu_count()) as p:
#     p_kn.append(list(p.map(math.log, range(1, 10000))))

# with ProcessPoolExecutor() as executor:
#     p_kn.append(list(executor.map(math.log, range(1, 10000))))

In [ ]:
import os
import time
import heapq as hq
import numpy as np
import pandas as pd
from tqdm import tqdm
import pdb

class Collaborative_Filtering:
    def __init__(self, save_topk: int):
        self.movie_similary = dict()
        self.save_topk = save_topk
        self.data = None
        self.group_dict = dict()    # 每部电影被哪些用户看过
        self.score_dict = dict()    # 每部电影被哪些用户看过相应的评分
        self.user_dict = dict()     # 每个用户看过哪些电影


    def cosin_similary(self, m1, m2):
        common_user = set(self.group_dict[m1]).intersection(self.group_dict[m2])
        if len(common_user) < 2:
            return None

        rating_m1 = []
        for idx, user in enumerate(self.group_dict[m1]):
            if user in common_user:
                rating_m1.append(self.score_dict[m1][idx])

        rating_m2 = []
        for idx, user in enumerate(self.group_dict[m2]):
            if user in common_user:
                rating_m2.append(self.score_dict[m2][idx])
        
        arr_m1, arr_m2 = np.array(rating_m1), np.array(rating_m2)
        return np.dot(arr_m1, arr_m2) / (np.linalg.norm(arr_m1) * np.linalg.norm(arr_m2))

    def keep_topk_sim(self, heap_m, insert_m, sim):
        small_heap = self.movie_similary[heap_m]
        if len(small_heap) < self.save_topk:
            hq.heappush(small_heap, (sim, insert_m))
        else:
            if sim > small_heap[0][0]:
                hq.heapreplace(small_heap, (sim, insert_m))

    def train(self, data: pd.DataFrame):
        movie = list(set(data["movieId"]))
        for m in movie:
            self.movie_similary.update({m: []})

        for row in data.itertuples(index=False):
            if row.userId not in self.user_dict:
                self.user_dict.update({row.userId: [row.movieId]})
            else:
                self.user_dict[row.userId].append(row.movieId)
            
            if row.movieId not in self.group_dict:
                self.group_dict.update({row.movieId: [row.userId]})
                self.score_dict.update({row.movieId: [row.rating]})
            else:
                self.group_dict[row.movieId].append(row.userId)
                self.score_dict[row.movieId].append(row.rating)

        for i1, m1 in enumerate(movie):
            for i2 in range(i1 + 1, len(movie)):
                m2 = movie[i2]
                sim = self.cosin_similary(m1, m2)
                if sim is not None:
                    self.keep_topk_sim(m1, m2, sim)
                    self.keep_topk_sim(m2, m1, sim)
            
            percent = i1 / len(movie) * 100
            print(f"\rProgress: {percent:6.2f}%", end="")



    def prediection(self, user_id, movie_id):
        user_seen = self.user_dict[user_id]
        candidate = []
        for s, movie in self.movie_similary[movie_id]:
            if movie in user_seen:
                for idx, u in enumerate(self.group_dict[movie]):
                    if u == user_id:
                        candidate.append((movie, s, self.score_dict[idx]))
                        break
        pred = sum(item[1] * item[2] for item in candidate) / sum(item[1] for item in candidate)
        return pred


df = pd.read_csv("C:\\vscode_project\\data_mining_homework\\data\\ml-latest-small\\ratings.csv")
df = df.sort_values("movieId")
cf = Collaborative_Filtering(100)
cf.train(df)

> c:\users\a7643\appdata\local\temp\ipykernel_16440\3899906307.py(68)train()

> c:\users\a7643\appdata\local\temp\ipykernel_16440\3899906307.py(69)train()

> c:\users\a7643\appdata\local\temp\ipykernel_16440\3899906307.py(64)train()

> c:\users\a7643\appdata\local\temp\ipykernel_16440\3899906307.py(65)train()

*** NameError: name 'common_user' is not defined
*** NameError: name 'common_user' is not defined
*** NameError: name 'common_user' is not defined


In [2]:
movie_id, user_id = 1, 1

user_seen = cf.user_dict[user_id]
candidate = []
for s, movie in cf.movie_similary[movie_id]:
    if movie in user_seen:
        for idx, u in enumerate(cf.group_dict[movie]):
            if u == user_id:
                candidate.append((movie, s, cf.score_dict[idx]))
                break
print(cf.movie_similary[193609])
# pred = sum(item[1] * item[2] for item in candidate) / sum(item[1] for item in candidate)


# pred = cf.prediection(1, 1)
# print(pred)

[(1.0, 10), (1.0, 34), (1.0, 50), (1.0, 593), (1.0, 293), (1.0, 296), (1.0, 318), (1.0, 1391), (1.0, 608), (1.0, 2712), (1.0, 2959), (1.0, 858), (1.0, 3623), (1.0, 1136), (1.0, 1213), (1.0, 3863), (1.0, 3911), (1.0, 3969), (1.0, 1676), (1.0, 4034), (1.0, 4223), (1.0, 4306), (1.0, 4446), (1.0, 3535), (1.0, 3578), (1.0, 4973), (1.0, 3717), (1.0, 3751), (1.0, 3752), (1.0, 3793), (1.0, 5952), (1.0, 6016), (1.0, 3897), (1.0, 6377), (1.0, 3948), (1.0, 6539), (1.0, 3977), (1.0, 4016), (1.0, 4018), (1.0, 7143), (1.0, 7254), (1.0, 7373), (1.0, 4239), (1.0, 33615), (1.0, 4367), (1.0, 8360), (1.0, 4447), (1.0, 4638), (1.0, 4816), (1.0, 4886), (1.0, 4963), (1.0, 37729), (1.0, 33794), (1.0, 4993), (1.0, 4995), (1.0, 5349), (1.0, 5418), (1.0, 5444), (1.0, 5502), (1.0, 33679), (1.0, 5669), (1.0, 102125), (1.0, 5956), (1.0, 33004), (1.0, 6157), (1.0, 6281), (1.0, 6373), (1.0, 34162), (1.0, 39183), (1.0, 6534), (1.0, 6537), (1.0, 34319), (1.0, 6541), (1.0, 6711), (1.0, 6754), (1.0, 164179), (1.0, 6863)

In [ ]:
qqq = []

hq.heappush(qqq, 3)
hq.heappush(qqq, 4)
hq.heappush(qqq, 1)
print(qqq)